## 1 Introduction

During the whole class, we have fully discussed the methods and tools to use location data to explore a geographical location. After deeply researching the neighborhoods and venues in Toronto, an idea comes to my mind.

  For a student who major in finance, just think about these two most famous cities in the world--New York and London. They are both advanced, well-known, contained big malls and high-class restaurants.But except for their location, are they really similar? We all know that when exploring a city, especially when we are travelling, we definitely want to know whether this city contains some unique bakerys, coffee shops, distribution centers,etc. And are these two different cities have some multicultural features? And if you decide to move to one of them, which city is fit for your life style?
  
  Hence, I decide to research two cities above standing on citizens' side. As citizens, they must care about common venues in their city and their distributions. The buildings in a city is what can best explain its culture. That's why I choose to explore city through maps. Also, based on my preference of New York, I will continue to explore every distinct of this city, and give advice on where to open a new kind of store to fit the new shopping complexes and malls there.
  
  Thanks for the convenience use of API in Foursquare, I can attain the detailed venues catalog in different cities. Also, some package in Python, like geopy, provides me some location information. Finally, I also downloaded some datasets from Internet,like Wiki's html. That's all the data source I will be using to solve the problem above.

## 2. Download and Explore Dataset

In [31]:
!conda install -c conda-forge lxml --yes 

Solving environment: ...working... done

# All requested packages already installed.



In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [6]:
import requests
import random
from lxml import etree

 Firstly, I downloaded data from Wikipedia, which contains London’s borough and post code. And I used package geopy (it’s very unstable) to produce its latitude and longitude. Also, I already owned data of newyork city.

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_areas_of_London' 
london= pd.read_html(url,encoding='utf-8')[1]
london.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [32]:
london['Borough'] = london[london.columns[1]].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
london=london.drop('London\xa0borough',axis=1)
london.head()

,Location,Post town,Postcode district,Dial code,OS grid ref,Borough
0,Abbey Wood,LONDON,SE2,020,TQ465785,"Bexley, Greenwich"
1,Acton,LONDON,"W3, W4",020,TQ205805,"Ealing, Hammersmith and Fulham"
2,Addington,CROYDON,CR0,020,TQ375645,Croydon
3,Addiscombe,CROYDON,CR0,020,TQ345665,Croydon
4,Albany Park,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728,Bexley


In [35]:
#Where the Postcode are more than one, the postcodes are spread to multi-rows and assigned the same values from the other columns
london = london.drop('Postcode\xa0district', axis=1).join(london['Postcode\xa0district'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postcode'))

In [68]:
dflondon = london[['Location', 'Borough', 'Postcode','Post town']].reset_index(drop=True)
dflondon=dflondon[dflondon['Post town'].str.contains('LONDON')]
dflondon = dflondon[['Location', 'Borough', 'Postcode']].reset_index(drop=True)
dflondon.head()

^C
imported


,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Acton,"Ealing, Hammersmith and Fulham",W3
2,Acton,"Ealing, Hammersmith and Fulham",W4
3,Aldgate,City,EC3
4,Aldwych,Westminster,WC2


In [94]:
# Geocoder starts here
from time import sleep
geolocator = Nominatim(user_agent="to_explorer")
la1=[]
lo1=[]
for i in range(len(dflondon)):
    ll=dflondon.at[i,'Postcode']+',London'
    location = geolocator.geocode(ll,timeout=None)
    latitude = location.latitude
    la1.append(latitude)
    longitude = location.longitude
    lo1.append(longitude)
    sleep(1)

C:\Users\44089\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: `timeout=None` has been passed to a geocoder call. Using default geocoder timeout. In geopy 2.0 the behavior will be different: None will mean "no timeout" instead of "default geocoder timeout". Pass geopy.geocoders.base.DEFAULT_SENTINEL instead of None to get rid of this warning.
  


GeocoderTimedOut: Service timed out

In [241]:
for i in range(len(la1)+1,len(dflondon)+1):
    ll=dflondon.at[i,'Postcode']+',London'
    location = geolocator.geocode(ll)
    latitude = location.latitude
    la1.append(latitude)
    longitude = location.longitude
    lo1.append(longitude)

KeyError: 381

In [256]:
dflondon.iloc[0,:]

Location            Abbey Wood
Borough     Bexley, Greenwich 
Postcode                   SE2
Name: 0, dtype: object

In [261]:
dflondon['Latitude']=la1
dflondon['Longitude']=lo1

In [7]:
#I already saved Processed data into table1.xlsx, so it can be used easily everytime I reconnect this page
dflondon=pd.read_excel('table1.xlsx')
dflondon.head()

,Location,Borough,Postcode,Latitude,Longitude
0,Abbey Wood,"Bexley, Greenwich",SE2,51.500304,0.120649
1,Acton,"Ealing, Hammersmith and Fulham",W3,38.194357,-81.368694
2,Acton,"Ealing, Hammersmith and Fulham",W4,38.194357,-81.368694
3,Aldgate,City,EC3,51.489334,-0.144055
4,Aldwych,Westminster,WC2,51.489334,-0.144055


 Also, I already owned data of newyork city.

In [14]:
import json
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [15]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [16]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

## 3. Methodology

#### Data Exploration

I drew the Stamen Terrain map of two cities and their boroughs. Since data of New York city is more detailed, you can see many paints on this island. And London is a plate with less forests.

In [17]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='Stamen Terrain')

# add markers to map
for lat, lng, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [14]:
address = 'London'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='Stamen Terrain')

# add markers to map
for lat, lng, borough in zip(dflondon['Latitude'], dflondon['Longitude'], dflondon['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

The geograpical coordinate of London are 51.5073219, -0.1276474.


As Asian, I am glad to see that the demography of Newham, London fits for my demand. Asian takes up 43.5% of the whole citizens there. Also, Harrow and Redbridge, take up 42.6% and 41.8% respectively. It’s such a high percentage that I think racial discrimination may not become problem in these local authority, regarding its multiple demography. Another city, New York, contains 9.4% of Asian in Manhattan, which is a considerable percentage in this multi-cultural city.

In [15]:
url2='https://en.wikipedia.org/wiki/Demography_of_London'
demo_london= pd.read_html(url2,encoding='utf-8')[4]
demo_london= demo_london.sort_values(by='Asian', ascending=False)
demo_london.head()

,Local authority,White,Mixed,Asian,Black,Other
24,Newham,29.0,4.5,43.5,19.6,3.5
13,Harrow,42.2,4.0,42.6,8.2,2.9
25,Redbridge,42.5,4.1,41.8,8.9,2.7
29,Tower Hamlets,45.2,4.1,41.1,7.3,2.3
17,Hounslow,51.4,4.1,34.4,6.6,3.6


In [36]:
url2='https://en.wikipedia.org/wiki/Demographics_of_New_York_City'
demo_ny= pd.read_html(url2,encoding='utf-8')[5]
demo_ny

,Jurisdiction,Population2000census,%white,%blackorAfricanAmerican,%Asian,%Other,%mixedrace,%Hispanic/Latinoof anyrace,Unnamed: 8,%Catholic,% notaffiliated,%Jewish,%Protestant,Estimateof % notreporting
0,NaN,NaN,Race,Race,Race,Race,Race,Ethnicity,NaN,Religious groups,Religious groups,Religious groups,Religious groups,Religious groups
1,Brooklyn,2465326,41.2,36.4,7.5,10.6,4.3,19.8,NaN,37,4,15,8,33
2,Queens,2229379,44.1,20.0,17.6,12.3,6.1,25.0,NaN,29,37,11,5,15
3,Manhattan,1537195,54.4,17.4,9.4,14.7,4.1,27.2,NaN,37,11,20,9,19
4,Bronx,1332650,29.9,35.6,3.0,25.7,5.8,48.4,NaN,44,14,6,5,29
5,Staten Island,443728,77.6,9.7,5.7,4.3,2.7,12.1,NaN,60,11,8,5,14
6,NYC Total,8008278,44.7,26.6,9.8,14.0,4.9,27.0,NaN,37,17,13,6,24
7,NY State,18976457,67.9,15.9,5.5,7.5,3.1,15.1,NaN,42,20,9,10,16
8,USA,281421906,75.1,12.3,3.6,6.5,2.4,12.5,NaN,22,37,2,23,12
9,"Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ...","Source: 2000 Census[63] American Indian, Nativ..."


In [37]:
demo_ny.drop([0,9,10],axis=0,inplace=True)
demo_ny= demo_ny.sort_values(by='%Asian', ascending=False)
demo_ny

,Jurisdiction,Population2000census,%white,%blackorAfricanAmerican,%Asian,%Other,%mixedrace,%Hispanic/Latinoof anyrace,Unnamed: 8,%Catholic,% notaffiliated,%Jewish,%Protestant,Estimateof % notreporting
6,NYC Total,8008278,44.7,26.6,9.8,14.0,4.9,27.0,NaN,37,17,13,6,24
3,Manhattan,1537195,54.4,17.4,9.4,14.7,4.1,27.2,NaN,37,11,20,9,19
1,Brooklyn,2465326,41.2,36.4,7.5,10.6,4.3,19.8,NaN,37,4,15,8,33
5,Staten Island,443728,77.6,9.7,5.7,4.3,2.7,12.1,NaN,60,11,8,5,14
7,NY State,18976457,67.9,15.9,5.5,7.5,3.1,15.1,NaN,42,20,9,10,16
8,USA,281421906,75.1,12.3,3.6,6.5,2.4,12.5,NaN,22,37,2,23,12
4,Bronx,1332650,29.9,35.6,3.0,25.7,5.8,48.4,NaN,44,14,6,5,29
2,Queens,2229379,44.1,20.0,17.6,12.3,6.1,25.0,NaN,29,37,11,5,15


In [ ]:
demo_ny= demo_ny.sort_values(by='%Asian', ascending=True)
demo_ny.head()

#### Analysing each neighborhood

In [8]:
CLIENT_ID = 'LXKBJ0B4G2SDDS2H03KIZM1YWRXNK5RZZEFHZYZDQR3LBOZL' # my Foursquare ID
CLIENT_SECRET = 'Z5AQJRWX1IXFTYK3SSEHBSWZ4GU3SBBVEUZCYG0M3DEKYNAN' # my Foursquare Secret
VERSION = '20200329'
LIMIT =100
radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=LXKBJ0B4G2SDDS2H03KIZM1YWRXNK5RZZEFHZYZDQR3LBOZL&client_secret=Z5AQJRWX1IXFTYK3SSEHBSWZ4GU3SBBVEUZCYG0M3DEKYNAN&v=20200329&ll=40.7127281,-74.0060152&radius=500&limit=100'

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e92d67669babe001beffc8c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 114,
  'suggestedBounds': {'ne': {'lat': 40.7172281045, 'lng': -74.00008952063419},
   'sw': {'lat': 40.7082280955, 'lng': -74.0119408793658}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57f0689d498e7d49d9189369',
       'name': 'The Bar Room at Temple Court',
       'location': {'address': '123 Nassau St',
        'lat': 40.7114477287544,
        'lng': -74.00680157032005,
        'labe

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

C:\Users\44089\anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

C:\Users\44089\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
#Explore Neighborhoods in two cities
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
london_venues = getNearbyVenues(names=dflondon['Borough'],
                                   latitudes=dflondon['Latitude'],
                                   longitudes=dflondon['Longitude'])

Bexley, Greenwich 
Ealing, Hammersmith and Fulham
Ealing, Hammersmith and Fulham
City
Westminster
Bromley
Islington
Islington
Islington
Barnet
Barnet
Enfield
Enfield
Wandsworth
Southwark
City
Richmond upon Thames
Barnet
Barnet
Islington
Wandsworth
Westminster
Bromley
Bromley
Newham
Newham
Newham
Ealing
Westminster
Lewisham
Camden
Southwark
Tower Hamlets
Bexley
Bexley
Bexley
City
Lewisham
Greenwich
Greenwich
Tower Hamlets
Camden
Haringey
Haringey
Tower Hamlets
Haringey
Barnet
Barnet
Brent
Lambeth
Lambeth
Lambeth
Lewisham
Tower Hamlets
Kensington and ChelseaHammersmith and Fulham
Brent
Barnet
Barnet
Southwark
Tower Hamlets
Camden
Tower Hamlets
Waltham Forest
Newham
Islington
Richmond upon Thames
Lewisham
Camden
Westminster
Greenwich
Kensington and Chelsea
Barnet
Westminster
Lewisham
Waltham Forest
Hounslow, Ealing, Hammersmith and Fulham
Brent
Barnet
Lambeth, Wandsworth
Islington
Barnet
Merton
Barnet
Barnet
Westminster
Barnet, Brent, Camden
Lewisham
Bexley
Haringey
Bromley
Bromley
Bromle

In [21]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

The difference between Newyork and London can be easily seen through their venues

In [22]:
london_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bexley, Greenwich",51.500304,0.120649,Lidl,51.496152,0.118417,Supermarket
1,"Bexley, Greenwich",51.500304,0.120649,Lakeside Complex,51.500789,0.123732,Pub
2,"Bexley, Greenwich",51.500304,0.120649,Southmere Lake,51.499354,0.123837,Lake
3,"Bexley, Greenwich",51.500304,0.120649,Abbey Wood Park,51.496284,0.120354,Park
4,"Bexley, Greenwich",51.500304,0.120649,The Birchwood,51.497774,0.115105,Pub


In [22]:
newyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


To be more specific, I counted the top10 venue category in each city.Statistics suggest that in London **hotel** is the commonest place, whereas in NewYork, there are 445 **Pizza Place**,which is what you can often see in this city.

In [47]:
nearby_venues_london = london_venues['Venue Category'].value_counts().to_frame(name='Count')
nearby_venues_london.head(10)

,Count
Hotel,1676
Café,1134
Coffee Shop,1127
Pub,1019
Italian Restaurant,817
Grocery Store,571
Bakery,546
Turkish Restaurant,487
Indian Restaurant,435
Pizza Place,388


In [43]:
nearby_venues_newyork = newyork_venues['Venue Category'].value_counts().to_frame(name='Count')
nearby_venues_newyork.head(10)

,Count
Pizza Place,445
Coffee Shop,315
Italian Restaurant,304
Deli / Bodega,269
Bar,231
Bakery,221
Chinese Restaurant,218
Grocery Store,192
Sandwich Place,185
Café,175


In this section, we compare the amount of venues in each neighborhoods.

In [45]:
lvcate=london_venues.groupby('Neighborhood').count()
lvcate= lvcate.sort_values(by='Venue Category', ascending=False)
lvcate

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Barnet,1520,1520,1520,1520,1520,1520
Hackney,1162,1162,1162,1162,1162,1162
Camden,1110,1110,1110,1110,1110,1110
Haringey,1067,1067,1067,1067,1067,1067
Lewisham,986,986,986,986,986,986
Tower Hamlets,826,826,826,826,826,826
Lambeth,822,822,822,822,822,822
Westminster,772,772,772,772,772,772
Islington,764,764,764,764,764,764


In [46]:
nycvat=newyork_venues.groupby('Neighborhood').count()
nycvat=nycvat.sort_values(by='Venue Category', ascending=False)
nycvat

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Murray Hill,148,148,148,148,148,148
Chelsea,105,105,105,105,105,105
Lenox Hill,100,100,100,100,100,100
Little Italy,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Village,100,100,100,100,100,100
Financial District,100,100,100,100,100,100


## Exploring NEW YORK

#### Analysing each neighborhood to find the proper location for business

In [23]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

newyork_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,O

In [24]:
newyork_nei = newyork_onehot.Neighborhood
newyork_onehot = newyork_onehot.drop('Neighborhood',axis=1)
newyork_onehot.insert(0,'Neighborhood',newyork_nei)
newyork_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,O

In [53]:
newyork_onehote=newyork_onehot.groupby('Neighborhood').count()

We count the number of venues in each neighborhoods to see its properity. It turns out that Murray Hill has the most recreation.

In [44]:
newyork_onehote= newyork_onehote.sort_values(by='Chinese Restaurant', ascending=False)
newyork_onehote.head(10)

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Ou

## Clustering
For this section, the neighbourhoods in New York will be clustered based on the processed data obtained above.
Map Visualization — Using the geopy library, the latitude and longitude values of NYU is obtained.

#### Regrouping and Category Statistics

In [46]:
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()

#print each neighborhood along with the top 5 most common venue
num_top_venues = 5

for hood in newyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = newyork_grouped[newyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                venue  freq
0         Pizza Place  0.15
1         Supermarket  0.11
2  Chinese Restaurant  0.07
3       Deli / Bodega  0.07
4         Gas Station  0.04


----Annadale----
                 venue  freq
0  American Restaurant  0.15
1          Pizza Place  0.15
2         Dance Studio  0.08
3        Train Station  0.08
4           Restaurant  0.08


----Arden Heights----
          venue  freq
0      Pharmacy   0.2
1  Home Service   0.2
2   Coffee Shop   0.2
3      Bus Stop   0.2
4   Pizza Place   0.2


----Arlington----
                 venue  freq
0             Bus Stop   0.2
1         Intersection   0.2
2        Deli / Bodega   0.2
3        Grocery Store   0.2
4  American Restaurant   0.2


----Arrochar----
                venue  freq
0       Deli / Bodega  0.11
1            Bus Stop  0.11
2  Italian Restaurant  0.11
3      Sandwich Place  0.05
4      Cosmetics Shop  0.05


----Arverne----
             venue  freq
0        Surf Spot  0.22
1   Sandwich Plac

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creating new dataframe
Then we create a new panda dataframe with 10 most common venues as shown below:

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Deli / Bodega,Chinese Restaurant,Spanish Restaurant,Donut Shop,Check Cashing Service,Grocery Store,Martial Arts Dojo,Pharmacy
1,Annadale,Pizza Place,American Restaurant,Dance Studio,Pub,Train Station,Sports Bar,Diner,Pharmacy,Food,Restaurant
2,Arden Heights,Pharmacy,Coffee Shop,Bus Stop,Home Service,Pizza Place,Field,Event Service,Event Space,Exhibit,Eye Doctor
3,Arlington,Deli / Bodega,American Restaurant,Intersection,Bus Stop,Grocery Store,Yoga Studio,Fish & Chips Shop,Eye Doctor,Factory,Falafel Restaurant
4,Arrochar,Italian Restaurant,Deli / Bodega,Bus Stop,Hotel,Supermarket,Outdoors & Recreation,Middle Eastern Restaurant,Mediterranean Restaurant,Pharmacy,Bagel Shop


#### Clustering of Neighbourhood

In [56]:
# set number of clusters
kclusters = 5

newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 1, 1, 4, 4, 4, 4, 4])

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

newyork_merged = neighborhoods

# merge newyork_grouped with newyork_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

newyork_merged.head() # check the last columns

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4.0,Pharmacy,Dessert Shop,Donut Shop,Laundromat,Gas Station,Caribbean Restaurant,Ice Cream Shop,Sandwich Place,Factory,Eye Doctor
1,Bronx,Co-op City,40.874294,-73.829939,4.0,Bus Station,Baseball Field,Park,Salon / Barbershop,Fast Food Restaurant,Pizza Place,Trail,Bagel Shop,Pharmacy,Grocery Store
2,Bronx,Eastchester,40.887556,-73.827806,4.0,Caribbean Restaurant,Bus Station,Diner,Deli / Bodega,Donut Shop,Metro Station,Seafood Restaurant,Fast Food Restaurant,Pizza Place,Platform
3,Bronx,Fieldston,40.895437,-73.905643,2.0,Plaza,Yoga Studio,Financial or Legal Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
4,Bronx,Riverdale,40.890834,-73.912585,4.0,Park,Bank,Playground,Food Truck,Bus Station,Gym,Home Service,Plaza,Event Space,Exhibit


In [71]:
newyork_merged= newyork_merged.sort_values(by='Cluster Labels', ascending=False)
newyork_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4.0,Pharmacy,Dessert Shop,Donut Shop,Laundromat,Gas Station,Caribbean Restaurant,Ice Cream Shop,Sandwich Place,Factory,Eye Doctor
61,Brooklyn,Williamsburg,40.707144,-73.958115,4.0,Bar,Coffee Shop,Grocery Store,Bagel Shop,Taco Place,Liquor Store,Lounge,Gym,Greek Restaurant,Tapas Restaurant
66,Brooklyn,Carroll Gardens,40.680540,-73.994654,4.0,Italian Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Bakery,Wine Shop,Bar,Gym / Fitness Center,Spa,Gift Shop
65,Brooklyn,Cobble Hill,40.687920,-73.998561,4.0,Coffee Shop,Bakery,Pizza Place,Bar,Playground,Yoga Studio,Deli / Bodega,Italian Restaurant,Cocktail Bar,Mediterranean Restaurant
64,Brooklyn,Brooklyn Heights,40.695864,-73.993782,4.0,Yoga Studio,Park,Deli / Bodega,Pharmacy,Pizza Place,Mexican Restaurant,Italian Restaurant,Gym,Bakery,American Restaurant
63,Brooklyn,Bedford Stuyvesant,40.687232,-73.941785,4.0,Pizza Place,Café,Bar,Coffee Shop,Thrift / Vintage Store,Cocktail Bar,Tiki Bar,Bus Station,Basketball Court,Park
62,Brooklyn,Bushwick,40.698116,-73.925258,4.0,Bar,Mexican Restaurant,Coffee Shop,Deli / Bodega,Bakery,Pizza Place,Thrift / Vintage Store,Discount Store,Vegetarian / Vegan Restaurant,French Restaurant
42,Bronx,Pelham Gardens,40.862966,-73.841612,4.0,Donut Shop,Bus Station,Pharmacy,Deli / Bodega,Spanish Restaurant,Boat or Ferry,Grocery Store,Chinese Restaurant,Bank,Sandwich Place
41,Bronx,Olinville,40.871371,-73.863324,4.0,Supermarket,Caribbean Restaurant,Laundromat,Convenience Store,Chinese Restaurant,Food,Fried Chicken Joint,Metro Station,Basketball Court,Deli / Bodega
40,Bronx,Castle Hill,40.819014,-73.848027,4.0,Bank,Diner,Pharmacy,Market,Pizza Place,Baseball Field,Food Court,Food Stand,French Restaurant,Event Service


In [72]:
#delete the NaN rows
newyork_merged.drop([207,257],inplace=True)

**Visualizing the Resulting Clusters** — To visualize the clusters, we have the following:

In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

It can be clearly seen that cluster 4 takes up for most locations,which means most of the New York Neighborhoods is similar, containing some pubs,restaurants,coffee shops,etc.

In [74]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0.0,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
183,Jamaica Estates,Eye Doctor,Intersection,Bus Station,Indian Restaurant,Yoga Studio,Financial or Legal Service,Event Space,Exhibit,Factory,Falafel Restaurant
179,Neponsit,Beach,Bar,Fish & Chips Shop,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
178,Rockaway Beach,Beach,Food Truck,Seafood Restaurant,Ice Cream Shop,BBQ Joint,Bus Station,Arepa Restaurant,Latin American Restaurant,Beach Bar,New American Restaurant
172,Breezy Point,Trail,Beach,Monument / Landmark,Yoga Studio,Financial or Legal Service,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm
85,Sea Gate,Beach,Bus Line,Bus Station,Spa,Lighthouse,Yoga Studio,Financial or Legal Service,Eye Doctor,Factory,Falafel Restaurant
302,Hammels,Beach,Deli / Bodega,Dog Run,Bus Station,Bus Stop,Café,Gym / Fitness Center,Fast Food Restaurant,Fried Chicken Joint,Shoe Store
212,Oakwood,Bus Station,Bar,Lawyer,Yoga Studio,Fish & Chips Shop,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm
305,Fox Hills,Sandwich Place,Bus Station,Bus Stop,Bar,Yoga Studio,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm


The individual clsuters dataframe can be obtained by using the following code (better to run each cluster individually:

In [75]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1.0,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
198,New Brighton,Bus Stop,Deli / Bodega,Park,Playground,Bowling Alley,Discount Store,Yoga Studio,Factory,Falafel Restaurant,Farm
32,Van Nest,Deli / Bodega,Pizza Place,Hookah Bar,Yemeni Restaurant,BBQ Joint,Supermarket,Food Truck,Middle Eastern Restaurant,Bus Station,Cosmetics Shop
264,Utopia,Deli / Bodega,Ice Cream Shop,South American Restaurant,Pizza Place,Playground,Donut Shop,Bakery,Spa,History Museum,Automotive Shop
39,Edgewater Park,Italian Restaurant,Deli / Bodega,Pizza Place,Asian Restaurant,Sports Bar,Park,Chinese Restaurant,Bar,Donut Shop,Coffee Shop
256,Randall Manor,Bus Stop,Deli / Bodega,Park,Pizza Place,Playground,Yoga Studio,Filipino Restaurant,Exhibit,Eye Doctor,Factory
171,Broad Channel,Deli / Bodega,Other Nightlife,Dive Bar,Sporting Goods Shop,Pizza Place,Yoga Studio,Financial or Legal Service,Exhibit,Eye Doctor,Factory
252,Shore Acres,Deli / Bodega,Italian Restaurant,Bus Stop,Intersection,Bar,Bagel Shop,Restaurant,Concert Hall,Pizza Place,Chinese Restaurant
89,Ocean Hill,Deli / Bodega,Grocery Store,Southern / Soul Food Restaurant,Supermarket,Playground,Chinese Restaurant,Dry Cleaner,Pizza Place,Bakery,Donut Shop
190,Belle Harbor,Beach,Deli / Bodega,Pub,Spa,Donut Shop,Bakery,Bagel Shop,Mexican Restaurant,Pharmacy,Chinese Restaurant
270,Mount Hope,Deli / Bodega,Grocery Store,Mexican Restaurant,Pizza Place,Donut Shop,Video Game Store,Spanish Restaurant,Sandwich Place,Ice Cream Shop,Metro Station


In [76]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2.0,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,Yoga Studio,Financial or Legal Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
303,Bayswater,Park,Playground,Yoga Studio,Filipino Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm
27,Clason Point,Park,Bus Stop,South American Restaurant,Boat or Ferry,Pool,Grocery Store,Yoga Studio,Filipino Restaurant,Eye Doctor,Factory
203,Todt Hill,Park,Yoga Studio,Event Service,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
192,Somerville,Park,Yoga Studio,Event Service,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [77]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 3.0,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
238,Butler Manor,Baseball Field,Pool,Bus Stop,Convenience Store,Financial or Legal Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
76,Mill Island,Pool,Fish & Chips Shop,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [78]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 4.0,newyork_merged.columns[[1] + list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Dessert Shop,Donut Shop,Laundromat,Gas Station,Caribbean Restaurant,Ice Cream Shop,Sandwich Place,Factory,Eye Doctor
61,Williamsburg,Bar,Coffee Shop,Grocery Store,Bagel Shop,Taco Place,Liquor Store,Lounge,Gym,Greek Restaurant,Tapas Restaurant
66,Carroll Gardens,Italian Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Bakery,Wine Shop,Bar,Gym / Fitness Center,Spa,Gift Shop
65,Cobble Hill,Coffee Shop,Bakery,Pizza Place,Bar,Playground,Yoga Studio,Deli / Bodega,Italian Restaurant,Cocktail Bar,Mediterranean Restaurant
64,Brooklyn Heights,Yoga Studio,Park,Deli / Bodega,Pharmacy,Pizza Place,Mexican Restaurant,Italian Restaurant,Gym,Bakery,American Restaurant
63,Bedford Stuyvesant,Pizza Place,Café,Bar,Coffee Shop,Thrift / Vintage Store,Cocktail Bar,Tiki Bar,Bus Station,Basketball Court,Park
62,Bushwick,Bar,Mexican Restaurant,Coffee Shop,Deli / Bodega,Bakery,Pizza Place,Thrift / Vintage Store,Discount Store,Vegetarian / Vegan Restaurant,French Restaurant
42,Pelham Gardens,Donut Shop,Bus Station,Pharmacy,Deli / Bodega,Spanish Restaurant,Boat or Ferry,Grocery Store,Chinese Restaurant,Bank,Sandwich Place
41,Olinville,Supermarket,Caribbean Restaurant,Laundromat,Convenience Store,Chinese Restaurant,Food,Fried Chicken Joint,Metro Station,Basketball Court,Deli / Bodega
40,Castle Hill,Bank,Diner,Pharmacy,Market,Pizza Place,Baseball Field,Food Court,Food Stand,French Restaurant,Event Service


## Results
The following are the highlights of the 5 clusters above:
1. Restaurants,Coffee shops,Parks are popular in the NYU.
2. The whole city remains similar lifestyle,so the venues like pub and food store is common everywhere.Only some places near the sea look different.
3. Although Pizza Place becomes most familiar venues, you can hardly see them in some distinct of NYU.
4. Although, the Clusters have variations, a very visible presence is the predominance of different restaurants.And it is obvious that in the area which Asian takes up high percentage, Chinese Restaurant(Asian Restaurant) is likely to be seen.

## 4 Discussion and Conclusion
It is very important to note that Clusters 4 is the most viable clusters to create a restaurant,and in the neighborhood 'Manhattan',Asian restaurant is easy to succeed.

Of course,if you want to open a restaurant which contains the element'unique','view of the sea',I strongly recommend you the location of Cluster 0.

In conclusion, this project would have had better results if there were more data in terms of crime data within the area, traffic access and allowance of more venues exploration with the Foursquare (limited venues for free calls).

Also, getting the ratings and feedback of the current restaurants within the clusters would have helped in providing more insight into the best location.